### Year 4 Topology 5 - Limit Checking - Overloading, Voltage limits

In [1]:
import pandas as pd
import altair as alt

##### Loading

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [2]:
list_all = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
list_gen_wind = [0,40,75] # for 3 wind gust scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for gen_wi in list_gen_wind:
            for lsc in list_lsc:
                file_csv_out = 'load\savnw_sol_' + str(gen) +'_hy_' + str(gen_hy) +  '_wi_' + str(gen_wi) + '_' + str(lsc) + '_l.csv'
                data_load = pd.read_csv(file_csv_out,header=1, skiprows=0)
                data_fil = data_load
                data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
                data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')
                data_fil['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + 'Hydro = ' + str(gen_hy) + ' MW, ' + 'Wind = ' + str(gen_wi) + ' MW, '  + lsc.upper()
                list_all.append(data_fil)
data_all = pd.concat(list_all).reset_index(drop=True)
data_all = data_all[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID',
       'Non Metered Bus', ' Branch I', 'Branch Flow MW', 'Branch Flow MVAR',
       'Branch MVA', 'MW Loss', 'MVAR Loss', 'Rate', '%I', '%MVA',
       'From Bus V', 'To Bus V']]

##### Line and Transformer Loading

Transformer and branch loading are categorised to following brackets based on the range to which it falls into 
* Loading less than 25%
* Loading 25% to 50%
* Loading 50% to 70%
* Loading 70% to 80%
* Loading 80% to 85%
* Loading 85% to 90%
* Loading 90% to 95%
* Loading 95% to 100%
* Loading 100% and above

Separating transformer and line branch loadings

In [3]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)

Defining loading brackets for transformer and line branches 

In [4]:
data = transformer_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch Flow MVAR','Branch MVA', 'Rate','%MVA']]
data['Loading'] = ''
data.loc[(data['%MVA'] >100), 'Loading'] = '%MVA>100'
data.loc[(data['%MVA'] <100), 'Loading'] = '95<%MVA<100'
data.loc[(data['%MVA'] <95), 'Loading'] = '90<%MVA<95'
data.loc[(data['%MVA'] <90), 'Loading'] = '85<%MVA<90'
data.loc[(data['%MVA'] <85), 'Loading'] = '80<%MVA<85'
data.loc[(data['%MVA'] <80), 'Loading'] = '70<%MVA<80'
data.loc[(data['%MVA'] <70), 'Loading'] = '50<%MVA<70'
data.loc[(data['%MVA'] <50), 'Loading'] = '25<%MVA<50'
data.loc[(data['%MVA'] <25), 'Loading'] = '%MVA<25'

<ipython-input-4-7f16dd71236d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Loading'] = ''


In [5]:
data_l = branch_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch MVA', 'Rate','%I']]
data_l['Loading'] = ''
data_l.loc[(data_l['%I'] >100), 'Loading'] = '%I>100'
data_l.loc[(data_l['%I'] <100), 'Loading'] = '95<%I<100'
data_l.loc[(data_l['%I'] <95), 'Loading'] = '90<%I<95'
data_l.loc[(data_l['%I'] <90), 'Loading'] = '85<%I<90'
data_l.loc[(data_l['%I'] <85), 'Loading'] = '80<%I<85'
data_l.loc[(data_l['%I'] <80), 'Loading'] = '70<%I<80'
data_l.loc[(data_l['%I'] <70), 'Loading'] = '50<%I<70'
data_l.loc[(data_l['%I'] <50), 'Loading'] = '25<%I<50'
data_l.loc[(data_l['%I'] <25), 'Loading'] = '%I<25'

<ipython-input-5-abcf08d21c83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_l['Loading'] = ''


##### Histogram Transformer Branch Loading 

A histogram for each scenario is plotted to check the no of transformer branches in the above defined loading bracket 

In [6]:
alt.Chart(data).mark_bar().encode(
    alt.Y('Loading',scale = alt.Scale(domain=['%MVA>100','95<%MVA<100','90<%MVA<95','85<%MVA<90','80<%MVA<85','70<%MVA<80','50<%MVA<70','25<%MVA<50','%MVA<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the transformer branch loading falls into a bracket of loading between 25 to 50 percentage. 

##### Histogram Line Branch Loading 

A histogram is plotted to check the no of line branches in the above defined loading bracket 

In [7]:
alt.Chart(data_l).mark_bar().encode(
    alt.Y('Loading', scale =alt.Scale(domain=['%I>100','95<%I<100','90<%I<95','85<%I<90','80<%I<85','70<%I<80','50<%I<70','25<%I<50','%I<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the line branch loading falls into a bracket of loading between 25% to 50%

In the following section the lines and transformer branches with loading greater than 80% is filtered 

##### Checking transformer loading greater than 80% 

In [8]:
transformer_overload  = transformer_loading[transformer_loading['%MVA'] > 80.0].reset_index(drop=True)
transformer_overload['Branch'] = transformer_overload['From Bus'].astype(str) + ' ' + transformer_overload['From Bus Ext'] + ' to ' + \
transformer_overload['To Bus'].astype(str) + ' '+ transformer_overload['To Bus Ext'] + ' ' + transformer_overload['Ckt ID'].astype(str)
list_branch = transformer_overload['Branch'].unique()
print(list_branch)

['3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1'
 '206 URBGEN      18.000 to 205 SUB230      230.00 1']


In [9]:
transformer_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4792,90.00,80.0,120.42,0.03,11.15,150.0,76.36,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
1,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, RLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4807,90.00,80.0,120.42,0.03,11.23,150.0,76.61,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
2,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",206,URBGEN 18.000,205,SUB230 230.00,1,206,29639,744.11,600.0,955.88,2.22,113.82,1150.0,80.35,83.12,18.000,230.00,206 URBGEN 18.000 to 205 SUB230 230....
3,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4886,90.00,80.0,120.41,0.03,11.59,150.0,77.86,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
4,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4791,90.00,80.0,120.42,0.03,11.15,150.0,76.35,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, RLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4848,90.00,80.0,120.41,0.03,11.42,150.0,77.27,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
86,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, HLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4934,90.00,80.0,120.41,0.03,11.83,150.0,78.64,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
87,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4797,90.00,80.0,120.42,0.03,11.18,150.0,76.44,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
88,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, RLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4850,90.00,80.0,120.41,0.03,11.43,150.0,77.29,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...


##### Transformer branch overload plot

In [10]:
alt.Chart(transformer_overload).mark_line().encode(
    alt.X('%MVA', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 700, width =400).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Checking branch loading greater than 80% 

In [11]:

branch_overload = branch_loading[branch_loading['%I'] > 80.0].reset_index(drop=True)
branch_overload['Branch'] = branch_overload['From Bus'].astype(str) + ' ' + branch_overload['From Bus Ext'] + ' to ' + \
branch_overload['To Bus'].astype(str) + ' '+ branch_overload['To Bus Ext'] + ' ' + branch_overload['Ckt ID'].astype(str)
list_branch = branch_overload['Branch'].unique()
print(list_branch)


['154 DOWNTN      230.00 to 203 EAST230     230.00 1'
 '203 EAST230     230.00 to 154 DOWNTN      230.00 1'
 '153 MID230      230.00 to 154 DOWNTN      230.00 1'
 '154 DOWNTN      230.00 to 153 MID230      230.00 1'
 '203 EAST230     230.00 to 205 SUB230      230.00 1'
 '203 EAST230     230.00 to 205 SUB230      230.00 2'
 '205 SUB230      230.00 to 203 EAST230     230.00 1'
 '205 SUB230      230.00 to 203 EAST230     230.00 2']


In [12]:
branch_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, LLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,402,-147.76,-50.04,156.01,1.01,10.09,200.0,80.18,78.00,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
1,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, RLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,411,-151.59,-48.78,159.25,1.06,10.58,200.0,82.02,79.62,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
2,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, RLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,404,152.65,49.68,160.53,1.06,10.58,200.0,80.51,80.27,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
3,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,434,-154.29,-57.02,164.49,1.18,11.77,200.0,86.56,82.24,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
4,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,426,155.46,59.46,166.45,1.18,11.77,200.0,84.86,83.22,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, LLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,442,169.95,45.49,175.93,1.26,12.65,200.0,88.23,87.96,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
147,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, RLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,466,-172.17,-46.54,178.35,1.37,13.65,200.0,92.98,89.17,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
148,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, RLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,460,173.54,50.73,180.80,1.37,13.65,200.0,91.66,90.40,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
149,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,491,-174.81,-55.02,183.26,1.51,15.10,200.0,97.82,91.63,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....


##### Transmission line branch overload plots

In [13]:
branch_fil = branch_overload[(branch_overload['Branch'] !='154 DOWNTN      230.00 to 203 EAST230     230.00 1') & (branch_overload['Branch'] !='203 EAST230     230.00 to 154 DOWNTN      230.00 1') ].reset_index(drop = True)
branch_fil_1 = branch_overload[(branch_overload['Branch'] =='154 DOWNTN      230.00 to 203 EAST230     230.00 1') | (branch_overload['Branch'] =='203 EAST230     230.00 to 154 DOWNTN      230.00 1') ].reset_index(drop = True)

In [14]:
alt.Chart(branch_fil).mark_line().encode(
    alt.X('%I', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 55, width =200).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

In [15]:
alt.Chart(branch_fil_1).mark_line().encode(
    alt.X('%I', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 550, width =200).facet('Branch',columns=3).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Bus Voltage

In [16]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
list_gen_wind = [0,40,75] # for 3 wind gust scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for gen_wi in list_gen_wind:
            for lsc in list_lsc:
                file_csv_out = 'voltage/savnw_sol_' + str(gen) +'_hy_' + str(gen_hy) +  '_wi_' + str(gen_wi) + '_' + str(lsc) + '_volt.csv'
                data_volt = pd.read_csv(file_csv_out,header=1, skiprows=0)
                data_volt['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + 'Hydro = ' + str(gen_hy) + ' MW, ' + 'Wind = ' + str(gen_wi) + ' MW, '  + lsc.upper()
                list_volt.append(data_volt)
data_all_volt = pd.concat(list_volt).reset_index(drop=True)
data_all_volt = data_all_volt[['Scenario','Bus Number', 'Bus Name', 'Area', 'Base Voltage', 'Bus Voltage(PU)','Bus Voltage(kV)']]

Similar to branch loading categorization into brackets of loadings, voltages are also categorised into brackets

In [17]:
data_v = data_all_volt
data_v['Voltage Range'] = ''
data_v.loc[(data_v['Bus Voltage(PU)'] >1.1), 'Voltage Range'] = 'Vbus>1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.1), 'Voltage Range'] = '1.05<Vbus<1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.05), 'Voltage Range'] = '1.0<Vbus<1.05'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.0), 'Voltage Range'] = '0.95<Vbus<1.0'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.95), 'Voltage Range'] = '0.95<Vbus<0.9'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.9), 'Voltage Range'] = 'Vbus<0.9'

##### Histogram Bus Voltages

A histogram is plotted to check the no of Buses in each of the above defined brackets 

In [18]:
alt.Chart(data_v).mark_bar().encode(
    alt.Y('Voltage Range', scale =alt.Scale(domain=['Vbus>1.1','1.05<Vbus<1.1','1.0<Vbus<1.05','0.95<Vbus<1.0','0.95<Vbus<0.9','Vbus<0.9'])),
    alt.X('count()')).properties(height = 100,width = 200).facet('Scenario',columns=6).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that most of the buses have voltages in the bracket of 1.0 PU and 1.05 PU

In the following section, the buses out of upper and lower normal operating range is filtered and reported 

##### Voltage Upper Limits 

In [19]:
data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)['Bus Number'].unique()

array([3018,  206,  211,  212], dtype=int64)

In [20]:
data_high = data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)
data_high

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, LLS",3018,CATDOG_G,5,13.8,1.051,14.508,1.05<Vbus<1.1
1,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, RLS",206,URBGEN,2,18.0,1.053,18.948,1.05<Vbus<1.1
2,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, LLS",3018,CATDOG_G,5,13.8,1.051,14.510,1.05<Vbus<1.1
3,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, RLS",206,URBGEN,2,18.0,1.053,18.945,1.05<Vbus<1.1
4,"Solar = 0 MW, Hydro = 400 MW, Wind = 75 MW, LLS",3018,CATDOG_G,5,13.8,1.051,14.508,1.05<Vbus<1.1
...,...,...,...,...,...,...,...,...
65,"Solar = 100 MW, Hydro = 600 MW, Wind = 0 MW, HLS",212,HYDRO_N,2,20.0,1.063,21.257,1.05<Vbus<1.1
66,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, HLS",211,HYDRO_G,2,20.0,1.063,21.253,1.05<Vbus<1.1
67,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, HLS",212,HYDRO_N,2,20.0,1.063,21.253,1.05<Vbus<1.1
68,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",211,HYDRO_G,2,20.0,1.063,21.260,1.05<Vbus<1.1


##### Upper voltage limit violation plot


In [21]:
alt.Chart(data_high).mark_line().encode(
    alt.X('Bus Voltage(PU)', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 200, width = 280).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Voltage Lower Limits

In [22]:
data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]['Bus Number'].unique()

array([154, 205], dtype=int64)

In [23]:
data_low = data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95].reset_index(drop=True)
data_low

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,"Solar = 0 MW, Hydro = 500 MW, Wind = 0 MW, HLS",154,DOWNTN,1,230.0,0.942,216.551,0.95<Vbus<0.9
1,"Solar = 0 MW, Hydro = 500 MW, Wind = 40 MW, HLS",154,DOWNTN,1,230.0,0.942,216.608,0.95<Vbus<0.9
2,"Solar = 0 MW, Hydro = 500 MW, Wind = 75 MW, HLS",154,DOWNTN,1,230.0,0.941,216.499,0.95<Vbus<0.9
3,"Solar = 0 MW, Hydro = 600 MW, Wind = 0 MW, HLS",154,DOWNTN,1,230.0,0.929,213.689,0.95<Vbus<0.9
4,"Solar = 0 MW, Hydro = 600 MW, Wind = 0 MW, HLS",205,SUB230,2,230.0,0.939,215.956,0.95<Vbus<0.9
5,"Solar = 0 MW, Hydro = 600 MW, Wind = 40 MW, HLS",154,DOWNTN,1,230.0,0.929,213.744,0.95<Vbus<0.9
6,"Solar = 0 MW, Hydro = 600 MW, Wind = 40 MW, HLS",205,SUB230,2,230.0,0.939,216.006,0.95<Vbus<0.9
7,"Solar = 0 MW, Hydro = 600 MW, Wind = 75 MW, HLS",154,DOWNTN,1,230.0,0.929,213.622,0.95<Vbus<0.9
8,"Solar = 0 MW, Hydro = 600 MW, Wind = 75 MW, HLS",205,SUB230,2,230.0,0.939,215.891,0.95<Vbus<0.9
9,"Solar = 50 MW, Hydro = 500 MW, Wind = 0 MW, HLS",154,DOWNTN,1,230.0,0.946,217.494,0.95<Vbus<0.9


##### Lower voltage limit violation plot

In [24]:
alt.Chart(data_low).mark_line().encode(
    alt.X('Bus Voltage(PU)', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 200, width = 280).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

#### Conclusion 

Year 4, Toplogy 5 limit violation check for overloading and voltage was carried out for the three load scenarios. It was seen that
1. Transformer and branch overloading of greater than 80% was reported for the following branches
   * Transformer 3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1
   * Transformer 206 URBGEN      18.000 to 205 SUB230      230.00 1
   * Branch 154 DOWNTN      230.00 to 203 EAST230     230.00 1
   * Branch 203 EAST230     230.00 to 154 DOWNTN      230.00 1
   * Branch 153 MID230      230.00 to 154 DOWNTN      230.00 1
   * Branch 54 DOWNTN      230.00 to 153 MID230      230.00 1
   * Branch 203 EAST230     230.00 to 205 SUB230      230.00 1
   * Branch 203 EAST230     230.00 to 205 SUB230      230.00 2
   * Branch 205 SUB230      230.00 to 203 EAST230     230.00 1
   * Branch 205 SUB230      230.00 to 203 EAST230     230.00 2
2. The buses violating the upper voltage limit are
   * 3018, 206, 211, 212
3. The buses violating the lower voltage limit are
   * 154, 205   